In [ ]:
import os, shutil
from matplotlib.image import imread
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from PIL import Image

from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
from tensorflow import keras
import tensorflow as tf
from keras.preprocessing import image

from tensorflow.keras.utils import plot_model

import pandas as pd

from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score

In [ ]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, 'C:/0_thesis/0_dataset-analysis')

from age_groups import *

In [ ]:
batch_size = 64 # !!

img_size = 256
x_col = 'filename'
y_col = ['age', 'gender']

In [ ]:
# ALL MODELS
ds_root = 'C:/0_thesis/dataset/'
ds_names = []
ds_names.append('UTKFace-wild')
ds_names.append('utkface-wild-05-pepper')
ds_names.append('utkface-wild-pepper')
ds_names.append('utkface-wild-gray')
ds_names.append('utkface-wild-05-gray-pepper')
ds_names.append('utkface-wild-gray-pepper')

models_names = ['rgb_no_alpha',
 'rgb_alpha05',
 'rgb_alpha1',
 'gray_no_alpha',
 'gray_alpha05',
 'gray_alpha1']

model_root = "C:/0_thesis/3_experiment/models/"

## ANALYSIS

In [ ]:
# Create dataframe to save metrics of all models
df = pd.DataFrame(models_names, columns = ['MODEL'])

In [ ]:
for i in range(len(models_names)):
    name = models_names[i]
    colormode = "rgb" if "rgb" in name else "grayscale"
    ds_path = os.path.join(ds_root, ds_names[i])
    model_path = os.path.join(model_root, name)
    
    validation_data = pd.read_csv(model_path+"/test_data.csv")
    val_datagen = ImageDataGenerator(rescale=1./255)

    val_generator = val_datagen.flow_from_dataframe(validation_data, 
                                                directory = ds_path, 
                                                x_col = x_col, 
                                                y_col = y_col, 
                                                target_size = (img_size, img_size),
                                                class_mode="multi_output",
                                                shuffle = False,
                                                batch_size = batch_size,
                                                color_mode = colormode)
    model = keras.models.load_model(model_path+"/model_4")
    # Evaluate
    #model.evaluate(val_generator)
    # Predict
    prediction = model.predict(val_generator)
    y_pred_age = np.round(prediction[0])
    y_pred_gender = np.round(prediction[1])

    y_pred_gender = y_pred_gender.astype('int')
    y_pred_age = y_pred_age.astype('int')
    validation_data.astype({'gender': 'int', 'age': 'int'})
    y_pred_groups = []
    for pred in y_pred_age:
        y_pred_groups.append(AgeGroups().getGroupFromAge(pred))
    # METRICS
    # GENDER
    row = df[df['MODEL'] == name].index
    df.loc[row, 'GENDER_PRECISION'] = np.round(precision_score(validation_data["gender"], y_pred_gender), 2)
    df.loc[row, 'GENDER_RECALL'] = np.round(recall_score(validation_data["gender"], y_pred_gender), 2)
    df.loc[row, 'GENDER_ACCURACY'] =  np.round(accuracy_score(validation_data["gender"], y_pred_gender), 2)
    df.loc[row, 'GENDER_F1'] =  np.round(f1_score(validation_data["gender"], y_pred_gender), 2)

    # AGE-GROUP
    df.loc[row, 'AGEGROUP_PRECISION'] = np.round(precision_score(validation_data["age-group"], y_pred_groups, average='micro'), 2)
    df.loc[row, 'AGEGROUP_RECALL'] = np.round(recall_score(validation_data["age-group"], y_pred_groups, average='micro'), 2)
    df.loc[row, 'AGEGROUP_ACCURACY'] = np.round(accuracy_score(validation_data["age-group"], y_pred_groups), 2)
    df.loc[row, 'AGEGROUP_F1'] = np.round(f1_score(validation_data["age-group"], y_pred_groups, average='micro'), 2)

In [ ]:
df

In [ ]:
df.to_csv(model_root+"/metrics.csv")